In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
import cv2
from skimage import io
import time

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GroupKFold, StratifiedKFold
import tqdm.notebook as tq
from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [ ]:

CFG = {
    
    'model_arch': 'tf_efficientnet_b3_ns',
    'img_size': 512,
    'valid_bs': 16,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

In [ ]:
df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')
df_torch = df.copy()
df_tf = df.copy()
 

In [ ]:
PATH = '/kaggle/input/cassava-leaf-disease-classification/test_images/'

> # **EFNet(PyTorch)**

In [ ]:
class CassavaImageClassifier(nn.Module):
    def __init__ (self, model_arch, n_class, pretrained=False):
        super().__init__ ()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

    def forward (self, x):
        x = self.model(x)
        return x

In [ ]:
class DiseaseDatasetInference(torch.utils.data.Dataset):

    def __init__ (self, df, transform=None, opt_label=True):
        self.df = df.reset_index(drop=True).copy()
        self.transform = transform
        self.opt_label = opt_label

        if self.opt_label:
            self.data = [(row['image_id'], row['label']) for _, row in self.df.iterrows()]

        else:
            self.data = [(row['image_id']) for _, row in self.df.iterrows()]

        self.data = np.asarray(self.data)
  
    def __len__(self):
        return len(self.data)

    def __getitem__ (self, index):
            # np.random.shuffle(self.data)
        if self.opt_label:
            image_path, label = self.data[index]    
        else:
            image_path = self.data[index]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        if self.opt_label == True:
            return (image, int(label))

        else:
            return image

In [ ]:
def get_inference_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
trained_model = torch.load('/kaggle/input/cassava-effecientnet-b3/model_6.pt', map_location=torch.device(CFG['device']))

In [ ]:
test_csv = df.copy()
test_csv['image_id'] = PATH + test_csv['image_id']

test_ds = DiseaseDatasetInference(test_csv, transform=get_inference_transforms(), opt_label=False)

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=CFG['valid_bs'], shuffle=False, pin_memory=False) 

In [ ]:
trained_model.eval()

preds_torch = []
with torch.no_grad():
    test_tqdm = valid_bar = tq.tqdm(test_loader, total=len(test_loader), desc="Testing", position=0, leave=True)
    for images in test_tqdm:
        images = images.to(CFG['device'])
    
        preds_torch.extend(trained_model(images).detach().cpu().numpy()) 

In [ ]:
torch_outcomes = pd.concat([df_torch['image_id'], pd.DataFrame(preds_torch)], axis=1).sort_values(['image_id'])

# **TF2(TF)**

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from PIL import Image

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/plantdiseaseresnet50/resnet50.h5')

In [ ]:
preds_tf = []

test_images = df_tf['image_id']
test_tqdm = valid_bar = tq.tqdm(test_images, total=len(test_images), desc="Testing", position=0, leave=True)
for image_path in test_tqdm:
    image = Image.open(PATH + image_path)
    image = image.resize((CFG['img_size'], CFG['img_size']))
    image = np.expand_dims(image, axis = 0)
    preds_tf.append(model.predict(image)[0])

In [ ]:
tf_outcomes = pd.concat([pd.DataFrame(test_images, columns=['image_id']), pd.DataFrame(preds_tf)], axis=1).sort_values(['image_id'])

# **Combine**

In [ ]:
final_preds = (torch_outcomes.drop('image_id', axis=1)*0.7 + tf_outcomes.drop('image_id', axis=1)*0.3).to_numpy().argmax(1)

In [ ]:
submit = pd.DataFrame({'image_id': torch_outcomes['image_id'].values, 'label': final_preds})
submit.to_csv('submission.csv', index=False)